In [5]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.request import Request, urlopen
import re

def get_rappler():
    
    try:
        # Listing down all the news articles about COVID

        rappler_covid_news = []
        page_link = '/previous-articles?filterMeta=coronavirus%20philippine%20updates'
        user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0)'

        while page_link != False:
            req = Request('https://www.rappler.com' + page_link, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'})
            content = urlopen(req).read()
            soup = BeautifulSoup(content)

            mydivs = soup.find("div", {'id':"article-finder-result"})

            mydivs = mydivs.find_all("div", {'class':"row"})

            rappler_covid_news += [url.a['href'] for url in mydivs] 
            time.sleep(6)

            try:
                page_link = soup.find("a", {'title':"Next"})['href']
            except:
                page_link = False

        rappler_df = pd.DataFrame(columns = ['source_id','date','category','title','author','text'])


        # Scraping all the articles in the list

        for article in rappler_covid_news:  
            user_agent = 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:55.0) '
            req = Request('https://www.rappler.com' + article, headers={'User-Agent': "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
                'Accept-Encoding': 'none',
                'Accept-Language': 'en-US,en;q=0.8',
                'Connection': 'keep-alive'})
            content = urlopen(req).read()
            soup = BeautifulSoup(content)

            try:
                article_id = ''

                date = soup.find("div", {'class':"published"}).text.strip()
                date = re.search("(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?).(\d{2}),.(\d{4})", date)
                date = date[0]

                category = 'nCov'
                title = soup.find("h1", {'class':"select-headline"}).text
                author = soup.find("a", {'class':"rappler-headline link"}).text.strip()
                text = soup.find("div", {'class':"cXenseParse"}).text
            except AttributeError:
                continue

            print(title, date)
            rappler_df = rappler_df.append(pd.Series([article_id,date,category, title,author, text], index = rappler_df.columns ), ignore_index=True)
            time.sleep(10)

        rappler_df.to_csv('rappler_tracker_scraped.csv', index = False)
    except:
        rappler_df.to_csv('new_rappler_tracker_scraped.csv', index = False)
    
    

In [6]:
get_rappler()

Cebu to start screening domestic passengers after coronavirus cases confirmed in Metro Manila March 09, 2020
Boracay combats coronavirus impact with discounts March 09, 2020
Cebu Pacific execs take pay cut to save jobs as coronavirus hurts profits March 09, 2020
PH airports to defer charging local airlines for a year due to coronavirus March 09, 2020
SBMA starts coronavirus disinfection protocol March 09, 2020
'No touch:' PSG bans close contact with Duterte amid coronavirus outbreak March 09, 2020
Cook Islands bans travelers from PH, 13 other countries over coronavirus scare March 09, 2020
Coronavirus seen to make Philippines miss 2020 GDP growth target March 09, 2020
Duterte declares state of public health emergency amid rise in coronavirus cases March 09, 2020
DOH: 4 new coronavirus cases admitted in hospitals in Metro Manila March 09, 2020
Qatar bans travelers from PH, 13 other countries over coronavirus fears March 09, 2020
‘No right to resist’ quarantine under state of public heal